In [17]:
import os
import pandas as pd

In [18]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_logician/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_logician/')

In [19]:
# selecting cell line
cline = 'NTERA2'  # HL60, MCF7, NTERA2

In [20]:
# reading in data
df = pd.read_table('data/raw/ctamdb_dpoa_' + cline + '.tsv')

In [21]:
# add the target protein (the protein that a phosphosite is on) to df
df['tprot'] = df['pst'].str.replace(r'\(.*$', "")
# add the phosphosite residue
df.loc[df['pst'].str.contains('\\(S'), 'residue'] = 's'
df.loc[df['pst'].str.contains('\\(T'), 'residue'] = 't'
df.loc[df['pst'].str.contains('\\(Y'), 'residue'] = 'y'

/var/folders/l3/vxqxpwf10_x9jp7g4_x9yy9m0000gn/T/ipykernel_70415/67434626.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tprot'] = df['pst'].str.replace(r'\(.*$', "")


In [22]:
df

,pst,perturbagen,fc,pval_eb,case,n_runs,n_ctr,n_trt,meansig_ctr,meansig_trt,sid_score,tprot,residue
0,AAAS(S495),AZD6482,-0.636767,0.001507,cross_batch,276,32.0,4.0,33.306748,33.308739,0.005174,AAAS,s
1,AAGAB(S310),AZD6482,-0.697781,0.166385,cross_batch,213,27.0,4.0,27.360502,27.524965,0.129019,AAGAB,s
2,AAGAB(S311),AZD6482,-0.121354,0.846852,cross_batch,237,28.0,4.0,28.493605,29.732421,0.391960,AAGAB,s
3,AAK1(S14),AZD6482,-0.900971,0.012035,cross_batch,269,32.0,4.0,28.480201,27.556952,0.070407,AAK1,s
4,AAK1(S21),AZD6482,-0.260125,0.349962,cross_batch,274,32.0,4.0,29.494494,29.302459,0.288801,AAK1,s
...,...,...,...,...,...,...,...,...,...,...,...,...,...
830998,USP32(T1326),Vemurafenib,0.000000,NaN,ctr_missing,23,2.0,0.0,25.802051,NaN,1.000000,USP32,t
830999,VAX2(T44),Vemurafenib,0.000000,NaN,ctr_missing,36,2.0,1.0,22.740180,25.060139,1.000000,VAX2,t
831000,WASHC2A(T6),Vemurafenib,0.000000,NaN,ctr_missing,23,3.0,0.0,21.821438,NaN,1.000000,WASHC2A,t
831001,ZBTB16(S256),Vemurafenib,0.000000,NaN,ctr_missing,14,2.0,0.0,25.931290,NaN,1.000000,ZBTB16,s


In [23]:
# export df to csv
df.to_csv(f'data/processed/facts/observations_mp_{cline}.csv', index=False)

In [24]:
# write Prolog file and assign down/up/unaffected based on p-value and log2fc
with open('models/facts/perturbs_' + cline + '_mp.pl', 'w') as file:
    for index, row in df.iterrows():
        if row['sid_score'] <= 0.05 and row['fc'] < 0:
            var1 = "perturbs('{}', '{}', '{}', {}, {}, {}, {}).".format(row['perturbagen'], row['pst'],
                                                                        row['tprot'], row['residue'], 'down',
                                                                        row['fc'], row['sid_score'])
        elif row['sid_score'] <= 0.05 and row['fc'] > 0:
            var1 = "perturbs('{}', '{}', '{}', {}, {}, {}, {}).".format(row['perturbagen'], row['pst'],
                                                                        row['tprot'], row['residue'], 'up',
                                                                        row['fc'], row['sid_score'])
        else:
            var1 = "perturbs('{}', '{}', '{}', {}, {}, {}, {}).".format(row['perturbagen'], row['pst'],
                                                                        row['tprot'], row['residue'], 'unaffected',
                                                                        row['fc'], row['sid_score'])
        file.write(var1 + '\n')